# Head Detection and Localization with Haar Cascades

A script for detecting and localizing heads in video clip frames using Haar Cascades.

The Haar Cascades used in this script are downloaded from [https://github.com/opencv/opencv/tree/master/data/haarcascades](https://github.com/opencv/opencv/tree/master/data/haarcascades).
Note that it's important that the *raw* version of the Haar Cascade be downloaded.

This script assumes that there is a `frames` folder with subfolders (e.g. `amadeus_frames`) containing film clip frames (e.g. `amadeus_00001.jpg`)  in the same directory as the script.

In [1]:
import os
from collections import defaultdict
import pickle

import cv2

In [2]:
def load_rgb_img(img_path):
    """
    Takes in img_path, the file path to an image.
    Reads the image at that path, and returns the RGB image.
    """
    img_raw = cv2.imread(img_path)
    
    # OpenCV imread reads in images in BGR color space, so must convert from BGR to RGB
    img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
    
    return img_rgb

In [3]:
def detect_feature_using_haar_cascade(img_path, haarcascade_filename, suffix):
    """
    Detects a feature in an image, using a Haar Cascade.
    
    Parameters:
    - img_path: the filepath to an image.
    - haarcascade_filename: a filename that encodes a Haar Cascade for a certain feature, ends with a ".xml" extension
                            (e.g. "haarcascade_frontalface_default.xml").
    - suffix: a suffix to append onto the Haar Cascade localization image folder name so that results from running multiple Haar Cascades don't override results from running a single Haar Cascade


    Writes a new version of the input image with a bounding box for each instance of the feature detected using the Haar Cascade.
    Returns a list of the head bounding box (x,y,w,h) coordinates detected by the cascade.
    """
    _, clip_frames_folder_name, img_name = img_path.split("/")
    
    img = load_rgb_img(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale to use Haar Cascade
    f_cascade = cv2.CascadeClassifier()
    f_cascade.load(cv2.samples.findFile(haarcascade_filename)) # the Haar Cascade file must be located in the same directory as this Jupyter Notebook
    detected = f_cascade.detectMultiScale(gray, 1.3, 5)
    img_head_localization = img.copy() # make a copy of original image so original image is retained while we draw bounding boxes on the copy

    # For each detected instance of the feature...
    for (x,y,w,h) in detected:
        # Draw a bounding box around the instance of the feature
        img_head_localization = cv2.rectangle(img_head_localization,(x,y),(x+w,y+h),(0,194,255),2)
    
    # If a directory for frames with head bounding boxes detected using Haar does not currently exist, make that directory
    # Path for single Haar Cascade results is head_localization_haar, path for 2 Haar Cascades is head_localization_haar_multiple_2, etc.
    head_localization_image_dir_path = "head_localization_haar" + suffix
    if not os.path.isdir(head_localization_image_dir_path):
        ! mkdir {head_localization_image_dir_path}
    
    # If a directory for current clip's head bounding box frames does not currently exist, make that directory
    clip_head_localization_image_dir_path = head_localization_image_dir_path + "/" + clip_frames_folder_name
    if not os.path.isdir(clip_head_localization_image_dir_path):
        ! mkdir {clip_head_localization_image_dir_path}

    # OpenCV imwrite expects BGR image, so convert from RGB to BGR color space before using OpenCV's imwrite function to save it
    cv2.imwrite(clip_head_localization_image_dir_path + "/" + img_name, cv2.cvtColor(img_head_localization, cv2.COLOR_RGB2BGR))
    return detected

**Run the block below to run head detection on all the clip frames in the `frames` folder, using Haar Cascades.**

Update `num_haar_cascades_to_use` variable depending on how many Haar Cascades you want to use for head detection (between 1 and 3).

In [4]:
num_haar_cascades_to_use = 2

# suffix will be used as suffix to append to folder names so that results from running multiple Haar Cascades don't override results from running a single Haar Cascade
if num_haar_cascades_to_use == 3:
    suffix = "_multiple_3"
elif num_haar_cascades_to_use == 2:
    suffix = "_multiple_2"
else:
    suffix = ""

# Get list of all files and directories in frames directory
frames_dir_path = "frames"
clips_list = os.listdir(frames_dir_path)

# For each clip's frames folder...
for clip_folder in clips_list:
    if clip_folder != ".DS_Store":
        # Get list of all files and directories in the clip's frames directory
        clip_frames_dir_path = frames_dir_path + "/" + clip_folder
        frames_list = os.listdir(clip_frames_dir_path)

        # Dictionary that will store head detection results using Haar Cascades
        # Keys are frame numbers, and values are whether or not the frame contains a head (0 if frame doesn't contain a head; 1 if it does).
        head_label_dict = {}
        
        # Dictionary that will store bounding box coordinates of detected heads
        # Keys are frame numbers, and values are list of (x,y,w,h) tuples for the head bounding boxes
        head_localization_dict = defaultdict(list)

        # For each frame...
        for frame_filename in frames_list:
            # We only want to detect heads in our frames, which are jpg files
            if frame_filename.endswith(".jpg"):
                frame_num = int(frame_filename.split("_")[-1].replace(".jpg", ""))
                # Detect if there's a head in the frame, using frontal face Haar Cascade
                detected_heads = detect_feature_using_haar_cascade(clip_frames_dir_path + "/" + frame_filename, "haarcascade_frontalface_default.xml", suffix)
                if len(detected_heads) > 0:
                    head_label_dict[frame_num] = 1
                    head_localization_dict[frame_num] = detected_heads
                else:
                    head_label_dict[frame_num] = 0
                    head_localization_dict[frame_num] = []
            
                if num_haar_cascades_to_use >= 2:
                    # If previous Haar Cascade(s) did not detect a head, try seeing if another Haar Cascade detects a head
                    if head_label_dict[frame_num] == 0:
                        detected_heads = detect_feature_using_haar_cascade(clip_frames_dir_path + "/" + frame_filename, "haarcascade_profileface.xml", suffix)
                        if len(detected_heads) > 0:
                            head_label_dict[frame_num] = 1   
                            head_localization_dict[frame_num] = detected_heads

                if num_haar_cascades_to_use >= 3:
                    # If previous Haar Cascade(s) did not detect a head, try seeing if another Haar Cascade detects a head
                    if head_label_dict[frame_num] == 0:
                        detected_heads = detect_feature_using_haar_cascade(clip_frames_dir_path + "/" + frame_filename, "haarcascade_frontalface_alt.xml", suffix)
                        if len(detected_heads) > 0:
                            head_label_dict[frame_num] = 1   
                            head_localization_dict[frame_num] = detected_heads

        # If a directory for head label dictionaries detected using Haar Cascades does not currently exist, make that directory
        # Path for single Haar Cascade is head_label_dicts_haar, path for 2 Haar Cascades is head_label_dicts_haar_multiple_2, etc.
        head_dict_haar_dir_path = "head_label_dicts_haar" + suffix
        if not os.path.isdir(head_dict_haar_dir_path):
            ! mkdir {head_dict_haar_dir_path}

        # Save (serialize) pickle of the Haar Cascade head detection result dictionary
        pickled_dict_filepath = head_dict_haar_dir_path + "/" + clip_folder.replace("_frames", "") + "_frame_to_head_dict" + ".pkl"
        with open(pickled_dict_filepath, "wb") as f:
            pickle.dump(head_label_dict, f)

        # Test loading (deserializing) pickled data to make sure pickled file saved correctly
        with open(pickled_dict_filepath, "rb") as f:
            pass
            # print(pickle.load(f))
            
        # If a directory for head bounding box dictionaries detected using Haar Cascades does not currently exist, make that directory
        # Path for single Haar Cascade is head_localization_dicts_haar, path for 2 Haar Cascades is head_localization_dicts_haar_haar_multiple_2, etc.
        head_localization_dict_haar_dir_path = "head_localization_dicts_haar" + suffix
        if not os.path.isdir(head_localization_dict_haar_dir_path):
            ! mkdir {head_localization_dict_haar_dir_path}

        # Save (serialize) pickle of the Haar Cascade head localization result dictionary
        pickled_dict_filepath = head_localization_dict_haar_dir_path + "/" + clip_folder.replace("_frames", "") + "_frame_to_head_box_coords_dict" + ".pkl"
        with open(pickled_dict_filepath, "wb") as f:
            pickle.dump(head_localization_dict, f)

        # Test loading (deserializing) pickled data to make sure pickled file saved correctly
        with open(pickled_dict_filepath, "rb") as f:
            pass
            # print(pickle.load(f))
